<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_30WMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-02-23 19:18:33


In [2]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=defc4591ef11dc9152d2468c38ae4580a1ee3a91e7ce74fc73d27479cbdf7f45
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
#mypf = mypf[~mypf['Criteria'].isin(['V40','V40N'])]
mypf = mypf[mypf['TFrame'] == 'W']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 10
      #mid_window = 50
      long_window = 30
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1wk')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['30_WMA'] = round(stock_df['Close'].ewm(span=long_window, adjust=False).mean(),0)
      stock_df['Dev%_30'] = round((stock_df['Close'] - stock_df['30_WMA'])*100/stock_df['30_WMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):
    #date = (datetime.date.today() + datetime.timedelta(days=0))
    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] > df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] > 60) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] < 60) ), -1.0, df_prec_dev['Signal'])

    df_prec_dev['Position'] = df_prec_dev['Signal'].diff()

    df_pos = df_prec_dev[(df_prec_dev['Position'] >= 1) | (df_prec_dev['Position'] <= -1)]
    df_pos = df_pos[df_pos['Signal'] != 0]
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop('Prev_Signal', axis=1, inplace=True)

    return df_pos.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_buy_sell = df_buy_sell.append(df_tmp)


    df_buy_sell['Position'] = df_buy_sell['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()


In [7]:
df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,,
2023-06-05 00:00:00+05:30,367.0,315.0,16.46,5PAISA.NS,692.60,278.40,31.77,88.80,90.0,1.0,Buy,V40N
2023-06-05 00:00:00+05:30,319.0,379.0,-15.78,CAMPUS.NS,598.50,240.15,32.92,87.50,26.0,-1.0,Sell,AR
2023-06-05 00:00:00+05:30,796.0,697.0,14.22,CAPLIPOINT.NS,1572.35,592.99,34.25,97.51,83.0,1.0,Buy,V40N
2023-06-05 00:00:00+05:30,1516.0,1412.0,7.38,MCX.NS,3829.90,1322.32,14.66,152.61,67.0,1.0,Buy,V40N
2023-06-05 00:00:00+05:30,885.0,810.0,9.23,MCDOWELL-N.NS,1166.05,743.98,18.93,31.79,91.0,1.0,Buy,V40N
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-12 00:00:00+05:30,451.0,453.0,-0.45,SIS.NS,605.18,320.65,40.64,34.20,49.0,-1.0,Sell,V40N
2024-02-12 00:00:00+05:30,224.0,233.0,-3.65,APEX.NS,402.28,190.56,17.81,79.19,47.0,-1.0,Sell,AR
2024-02-12 00:00:00+05:30,1786.0,1557.0,14.70,METROPOLIS.NS,3398.33,1200.40,48.78,90.29,64.0,1.0,Buy,VR


In [8]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Sell'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,,
2024-01-15 00:00:00+05:30,3732.0,3816.0,-2.21,DMART.NS,5323.75,3311.30,12.70,42.66,46.0,-1.0,Sell,AR
2024-01-15 00:00:00+05:30,4580.0,4651.0,-1.53,FINEORG.NS,7073.89,4129.51,10.91,54.46,49.0,-1.0,Sell,V40N
2024-01-22 00:00:00+05:30,7667.0,7914.0,-3.12,TATAELXSI.NS,10158.41,5912.75,29.67,32.49,45.0,-1.0,Sell,V40N
2024-01-22 00:00:00+05:30,1450.0,1464.0,-0.96,INDIGOPNTS.NS,2623.10,1006.05,44.13,80.90,48.0,-1.0,Sell,V40N
2024-01-22 00:00:00+05:30,512.0,518.0,-1.25,PRAJIND.NS,630.40,321.28,59.21,23.25,47.0,-1.0,Sell,AR
2024-01-22 00:00:00+05:30,1138.0,1161.0,-2.00,SFL.NS,1938.22,993.90,14.48,70.35,48.0,-1.0,Sell,V40N
2024-01-22 00:00:00+05:30,867.0,884.0,-1.97,HAPPSTMNDS.NS,1484.83,762.90,13.59,71.35,47.0,-1.0,Sell,AR
2024-01-22 00:00:00+05:30,141.0,142.0,-0.56,CUB.NS,239.46,119.00,18.66,69.59,50.0,-1.0,Sell,AR
2024-01-22 00:00:00+05:30,2389.0,2430.0,-1.70,LALPATHLAB.NS,4054.17,1799.04,32.78,69.72,47.0,-1.0,Sell,V40N


In [9]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Buy'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,,
2023-08-21 00:00:00+05:30,37.0,33.0,11.06,TRIDENT.NS,63.48,26.80,36.75,73.21,70.0,1.0,Buy,VR
2023-09-04 00:00:00+05:30,807.0,726.0,11.17,BLUESTARCO.NS,1293.75,684.35,17.94,60.30,62.0,1.0,Buy,AR
2023-09-04 00:00:00+05:30,16.0,15.0,7.03,INFIBEAM.NS,38.50,13.46,19.24,139.88,61.0,1.0,Buy,AR
2023-10-30 00:00:00+05:30,1320.0,1228.0,7.51,RADICO.NS,1725.25,1100.37,19.98,30.68,61.0,1.0,Buy,V40N
2023-11-13 00:00:00+05:30,584.0,545.0,7.23,SONACOMS.NS,780.79,408.54,43.04,33.61,63.0,1.0,Buy,AR
2023-11-13 00:00:00+05:30,2811.0,2369.0,18.67,CAMS.NS,3580.36,1998.84,40.65,27.35,69.0,1.0,Buy,VR
2023-11-20 00:00:00+05:30,675.0,628.0,7.49,LICI.NS,1076.70,529.76,27.43,59.50,65.0,1.0,Buy,V200
2023-11-27 00:00:00+05:30,4971.0,4612.0,7.78,BRITANNIA.NS,5338.45,4129.15,20.38,7.40,66.0,1.0,Buy,AR
2023-11-27 00:00:00+05:30,553.0,509.0,8.70,AARTIIND.NS,1099.51,449.40,23.12,98.72,64.0,1.0,Buy,VR


In [10]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [11]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)